In [0]:
matches_df = spark.read.csv("/FileStore/tables/matches.csv", header=True, inferSchema=True)
deliveries_df = spark.read.csv("/FileStore/tables/deliveries.csv", header=True, inferSchema=True)
deliveries_with_year_df = deliveries_df.join(
    matches_df,
    deliveries_df.match_id == matches_df.id,
    how="inner"
)
from pyspark.sql.functions import sum as spark_sum, avg, max as spark_max

runs_per_match_df = deliveries_with_year_df.groupBy(
    "season", "batting_team", "match_id"
).agg(
    spark_sum("total_runs").alias("total_runs_in_match")
)
average_score_pivot = runs_per_match_df.groupBy("batting_team").pivot("season").agg(
    avg("total_runs_in_match")
).orderBy("batting_team")

print(" Pivot Table: Average Score")
average_score_pivot.show(truncate=False)
max_score_pivot = runs_per_match_df.groupBy("batting_team").pivot("season").agg(
    spark_max("total_runs_in_match")
).orderBy("batting_team")

print(" Pivot Table: Max Score")
max_score_pivot.show(truncate=False)


 Pivot Table: Average Score
+---------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|batting_team               |2007/08           |2009              |2009/10           |2011              |2012              |2013              |2014              |2015              |2016              |2017              |2018              |2019              |2020/21           |2021              |2022              |2023              |2024              |
+---------------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+---------